# pandas による時系列データ処理入門（その2）

- 本ノートブックで扱う内容
    - ファイルの読み込み
    - 基準日を変えながら行うデータ処理
- 事前知識
    - `pandas` とは何か知っている
    - `pd.DataFrame` を少し触った事がある

---

# 0. 前準備

- ライブラリのインポート

In [1]:
import numpy as np
import pandas as pd
#
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

- 作業ディレクトリの変更
    - 詳細はサポートページのTIPsにある `colab.ipynb` 参照

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
# GoogleDrive内のColabNotebooksディレクトリへ移動
%cd "/content/drive/My Drive/Colab Notebooks/"

---

# 1. データファイルの読み込み

資料「pandas による時系列データ処理入門（その1）」では時系列データを自分で生成した。<br>
しかし、実際には何らかの方法で取得したデータを分析することが大半である。<br>
この資料を書いている時点では、手軽に扱える範囲のデータであればCSVファイル形式で配布されていることが多い。<br>
そこで、本資料ではCSV形式で与えられた時系列データの読み込み方法を解説する。

## 1.1. 時系列CSVデータの読み込み

まずは、サンプルデータ [sample_ts_data.csv](https://github.com/Masashi-Ieda/seminar_support/blob/main/ipynb/sample_ts_data.csv) を読み込んでみよう。<br>
（GitHub上にあるのでダウンロードして適当なフォルダに置くこと。）<br>
`pandas` で csv を読み込む場合は `read_csv()`を使うのであった。<br>
ナイーブに実行してみよう。

In [3]:
df = pd.read_csv('sample_ts_data.csv') 
df.head(3)

,date,A,B
0,2016-12-31,100.000000,100.000000
1,2017-01-31,99.740490,100.190601
2,2017-02-28,99.796436,100.107242


問題なく読み込めたが、このままだと `pandas` の時系列処理機能が全く使えない。<br>
これは日付を表しているはず `date` 列が `object` 型になっているためである。

In [4]:
df['date'].head(3)

0    2016-12-31
1    2017-01-31
2    2017-02-28
Name: date, dtype: object

つまり、ナイーブに読み込むのではなく<br>
`date`列が日付であることを明示して読み込む必要がある。<br>
これには読み込み時に `parse_dates=['列名']` と指定すれば良い

In [5]:
df = pd.read_csv('sample_ts_data.csv', parse_dates=['date'])
df['date'].head(3) 

0   2016-12-31
1   2017-01-31
2   2017-02-28
Name: date, dtype: datetime64[ns]

資料(その1) と同様に日付をindexとしたいのであれば<br>
`index_col='date'` のオプションを追加する。

In [6]:
df = pd.read_csv('sample_ts_data.csv', index_col='date', parse_dates=['date'])
# df = pd.read_csv('sample_ts_data.csv', index_col='date', parse_dates=True) # これでもOK
df.head(3)

,A,B
date,,
2016-12-31,100.000000,100.000000
2017-01-31,99.740490,100.190601
2017-02-28,99.796436,100.107242


なお、読み込み段階で日付変換する必要は必ずしもない。<br>
以下のようなやり方で読み込み後に変換をかけても良い

In [7]:
df = pd.read_csv('sample_ts_data.csv')
df['date'] = pd.to_datetime(df['date'])
# date 列をインデックスにしたいなら下を追加
df = df.set_index('date')
df.head(3)

,A,B
date,,
2016-12-31,100.000000,100.000000
2017-01-31,99.740490,100.190601
2017-02-28,99.796436,100.107242


## 1.2. 読み込みに前処理が必要な場合

ネットでは様々な時系列データが配布されているが<br>
ダウンロードしたデータがそのまま　`pandas`　で使えるケースは結構まれである。<br>
理由はいくつか考えられるが、<br>
「人間が見るだけの前提で、機械的に処理することを考えて作られていない」<br>
あたりが大きい様に感じる。（特に行政データに多くて頭が痛い）

そのままでは自動処理にかけられないデータを加工することを「前処理」という。<br>
ここでは、そのままでは `read_csv` を使えないデータに対して行う<br>
前処理の非常に簡単な例を紹介する。

### 1.2.1. 前処理が必要なデータ例

サンプルデータ [sample_ts_data_dirty.csv](https://github.com/Masashi-Ieda/seminar_support/blob/main/ipynb/sample_ts_data_dirty.csv) を例に説明しよう。
とりあえず `read_csv` で読みこんでみると

In [8]:
df_dirty = pd.read_csv('sample_ts_data_dirty.csv')
df_dirty.head(5)

,This is a dirty sample CSV for ts_data_intro
0,日本語はExcelで開けると文字化けする（はず） ; Japanese characters...
1,;A;B
2,2016-12-31;100.0;100.0
3,2017-01-31;99.74048964415707;100.19060082781897
4,2017-02-28;99.79643570468552;100.10724227001089


明らかに正常に読み込めていない。

おかしいと思った場合は、データをテキストエディタなどで開いて原因を探そう。<br>
今回の原因は主に3点である。

- 先頭2行に妙な文字列（今回の場合はコメント）がある。
- データの区切り文字が `,` ではなく `;` になっている。
- 日付に相当する列名が空白。

以上が解消されれば、1.1. で説明したコードで読み込むことができる。

### 1.2.2. テキストエディタで処理

慣れない内、あるいは単発で済むことが確定している場合は<br>
テキストエディタなどで置換・削除処理をしてしまう方が良い。<br>
`sample_ts_data.csv`と全く同じファイルにできるので、頑張って加工してみよう。

**注意点**

- **__絶対にExcelで処理をしてはいけない__**
    - 以下の例の様に CSVファイルが汚染されて `pandas` で扱えなくなる
        - 日付がシリアル値に置換される
        - 日本語が文字化けして保存される可能性あり
- おすすめテキストエディタ
    - Visual Studio Code
        - **超おすすめ**
        - 便利なプラグイン多数
        - プログラムも書きやすい
    - メモ帳
        - ただし、文字化けには対応できない
- 「正規表現」を覚えると、加工の自由度が格段に上がる
    - ただし、沼

### 1.2.3. コードのみで処理

今回のケース程度であれば、以下の様にコードのみで処理することもできる。<br>
特に1行目で使っている `read_csv` のオプション

- `sep` : 区切り文字の指定。デフォルトは `,` 。タブを指定したいときは `\t` とする。
- `skiprows` : 指定した行数だけ読み込みをスキップ。

は覚えておくととても便利。

In [9]:
df_dirty = pd.read_csv('sample_ts_data_dirty.csv', sep=';', skiprows=2)
df_dirty = df_dirty.rename(columns={df_dirty.columns[0]: 'date'})
df_dirty['date'] = pd.to_datetime(df_dirty['date'])
df_dirty.head(3)

,date,A,B
0,2016-12-31,100.000000,100.000000
1,2017-01-31,99.740490,100.190601
2,2017-02-28,99.796436,100.107242


### 1.2.4. 練習問題

[みずほ銀行のヒストリカルデータ提供ページ](https://www.mizuhobank.co.jp/market/historical.html)
の為替データを読み込んでみよう。<br>
（1.2.3. のコードをベースにすれば、多分すぐにできる。）

## 1.3. Excelファイルの読み込み

- `pandas` の `pd.read_excel()` で事足りるならOK。
- 事足りなくなった場合、別ライブラリを使う
    - `openpyxl`
    - `xlwings`

必要になったら追記します。

---

# 2. 移動平均 : 基準日を変えながら行うデータ処理

時系列データ $X = \{X_1,X_2, \cdots, X_N \}$ があるとする。<br>
このとき、基準日 $t$ から過去 $M$ 期間分のデータを使って平均値を $\bar{X}_t$と書くことにしよう：

$ \displaystyle \bar{X}_t := \frac{X_t + X_{t-1} + \cdots + X_{t-(M-1)}}{M} = \frac{1}{M} \sum_{i=0}^{M-1} X_{t-i}$

ここで定義した $\bar{X}_t$ を $X$ の $M$ 期間移動平均 (Moving Average) と呼ぶ。<br>

「移動」と呼ぶ理由はの意味は基準日 $t$ を移動させながら<br>
新しい時系列データ $\bar{X} = \{\bar{X}_t\}$ を作ることに由来する。<br>
また、移動平均は時系列データの平滑化方法の一種に分類される。<br>
（平滑化の意味は、下のグラフまで先回りしてみると分かりやすいかもしれない）


では、1.1.で読み込んだサンプルデータを使って $M=3$ とした場合の移動平均を求めてみよう。<br>
なお、サンプルが1ヶ月毎のデータなので、今回の移動平均を「3ヶ月移動平均」とも呼ぶ。

以下では移動平均の実装方法を解説する。<br>
ただ、2.1. はかなり冗長な説明なので、読み飛ばしても構わない。<br>
先を読んでわからなくなったら戻ってくるというスタンスが適当と考えている。<br>
逆にプログラミングに自信のない人は 2.1. からスタートすることをおすすめする。


## 2.1. プログラムを組む前に

### 2.1.1. $X_t$ のラベル $t$ とデータの日付の関係

- $X_t$ についているラベル $t$ は $X_1,X_2,\cdots$ と整数
- 計算基準日の $t$ は `2016-12-31`, `2017-3-31`, ... と日付

ここにギャップを感じるのであれば、一旦表にして整理してみると良い。

| 計算基準日 | 2017-3-31 | 2017-4-30 | 2017-5-31 |
|-------|-----------|-----------|-----------|
| t     | 2017-3-31 | 2017-4-30 | 2017-5-31 |
| t-1   | 2017-2-28 | 2017-3-31 | 2017-4-30 |
| t-2   | 2017-1-31 | 2017-3-31 | 2017-3-31 |


慣れないうちは、いちいち上記の表を書き出すようにすると<br>
ミスが減るのでトータルでは楽ができる。

※ 数式上もカレンダーで定義すると矛盾なく見えるかもしれないが<br>
時間間隔（今回は1ヶ月）などはデータによって異なる。<br>
抽象的に扱う方が応用範囲が広いので、頑張って慣れたほうが幸せ。



### 2.1.2. $\bar{X}_t$ の具体的な計算例

コードを書くときに重要なのは、コードを書く前に

**いくつか具体的な計算をしてみて、共通部分を見つけ出す**

ことをしておくことである。

最初3パターンの計算をしてみると、以下の表の様になる


| date       | A           | 移動平均        |
|------------|-------------|-------------|
| 2016/12/31 | 100         |             |
| 2017/1/31  | 99.74048964 |             |
| 2017/2/28  | 99.7964357  | 99.84564178 |
| 2017/3/31  | 99.58297033 | 99.70663189 |
| 2017/4/30  | 100.0420274 | 99.80714449 |
| 2017/5/31  | 100.4447953 | 100.0232644 |


この段階ではExcelを使うととても便利である。<br>
ただ、Excel だとデータサイズ（計算期間や計算対象の種類）が変わるだけで<br>
そのシートを修正する必要があるので、プログラムにしておくほうが後々楽。

## 2.2. ナイーブな実装過程

実装する過程を順を追って見ていこう。

初めに行うべきはデータのロードである。<br>
基本的には1.1. で行ったことと同じ様にすればOKだが<br>
実際の分析時には `df` など、何を表しているかわからない変数名は基本的に避けるべきである。

In [10]:
# 時系列データの読み込み。分析のときは名前をキチンとつける。
ts_data = pd.read_csv('sample_ts_data.csv', index_col='date', parse_dates=['date'])

次に、計算基準日を決定しよう。<br>
これには、ロードした`ts_data` の日付を使えば確実である。<br>
以下の様にして取り出そう。

In [11]:
# 計算基準日を抽出
#   3ヶ月経つまではデータが足りず移動平均が計算できない
calc_dates = ts_data.index[2:]

各計算基準日で使用するサンプルデータを取り出すための条件式を考えよう。<br>
前の資料に習うなら、開始日 `from_date` と 終了日 `to_date` を決めれば良い。<br>

- `to_date` : 計算基準日
- `from_date` : 計算基準日 - 2ヶ月

`- 2ヶ月` の部分は、`pd.DateOffset` という機能を使うと実装できる。

- 実装方法に悩んだらGoogleで検索してみよう。
    - 作成者も実装開始時に`pd.DateOffset`を覚えていたわけではない。
    - 「pandas datetime add months」と検索して使えそうな方法だったので使った。
- 他にも実装方法はある : `datetime.timedelta`を使う方法など。

In [12]:
for calc_date in calc_dates[:4]: # 出力の都合上、先頭4個の計算に制限
    # 検索条件を指定
    to_date = calc_date
    from_date = calc_date - pd.DateOffset(months=2)
    print('from_date = ', from_date, '   to_date = ', to_date)

from_date =  2016-12-28 00:00:00    to_date =  2017-02-28 00:00:00
from_date =  2017-01-31 00:00:00    to_date =  2017-03-31 00:00:00
from_date =  2017-02-28 00:00:00    to_date =  2017-04-30 00:00:00
from_date =  2017-03-31 00:00:00    to_date =  2017-05-31 00:00:00


出力結果を見る限り、良さそう。<br>
なので、データを絞り込んでみよう

In [13]:
for calc_date in calc_dates[:4]: # 出力の都合上、先頭4個の計算に制限
    # 検索条件を指定
    to_date = calc_date
    from_date = calc_date - pd.DateOffset(months=2)
    # データの絞り込み
    samples = ts_data[(ts_data.index >= from_date) & (ts_data.index <= to_date)]
    print(samples)

                     A           B
date                              
2016-12-31  100.000000  100.000000
2017-01-31   99.740490  100.190601
2017-02-28   99.796436  100.107242
                    A           B
date                             
2017-01-31  99.740490  100.190601
2017-02-28  99.796436  100.107242
2017-03-31  99.582970   98.833458
                     A           B
date                              
2017-02-28   99.796436  100.107242
2017-03-31   99.582970   98.833458
2017-04-30  100.042027   99.517338
                     A          B
date                             
2017-03-31   99.582970  98.833458
2017-04-30  100.042027  99.517338
2017-05-31  100.444795  99.657631


使いたいデータが出ていることが見て取れる。<br>
なので、平均を取る処理を加えて結果を表示させよう。

In [14]:
for calc_date in calc_dates[:4]: # 出力の都合上、先頭4個の計算に制限
    # 検索条件を指定
    to_date = calc_date
    from_date = calc_date - pd.DateOffset(months=2)
    # データの絞り込み
    samples = ts_data[(ts_data.index >= from_date) & (ts_data.index <= to_date)]
    # 平均値の計算 / pandas にやらせてしまう
    mean = samples.mean()
    # 結果の表示
    print(calc_date)
    print(mean)


2017-02-28 00:00:00
A     99.845642
B    100.099281
dtype: float64
2017-03-31 00:00:00
A    99.706632
B    99.710434
dtype: float64
2017-04-30 00:00:00
A    99.807144
B    99.486013
dtype: float64
2017-05-31 00:00:00
A    100.023264
B     99.336142
dtype: float64


これで実装はほぼ完了だが、コードが間違っていないか点検しておくことをおすすめする。<br>
今回の例であれば、最初3件程度について<br>
手計算(あるいはExcelなど)で計算した結果と一致するかチェックすると良い。<br>
**割と普通にミスするのでチェックは大事**

## 2.3.　もう少しスマートな実装例

2.2. よりもスマートな実装を考えてみよう。<br>
主な改善ポイントは以下の2点。

- 平均を計算するサンプルの絞り込み方法
- 結果の `pd.DataFrame` での出力方法

先に実装コードを示そう。

In [15]:
# M期間をパラメータにしておくと便利
M = 3
# 時系列データの読み込み。分析のときは名前をキチンとつける。
ts_data = pd.read_csv('sample_ts_data.csv', index_col='date', parse_dates=['date'])

# 計算対象日を抽出
#   Mヶ月経つまではデータが足りず移動平均が計算できない
calc_dates = ts_data.index[M-1:]
# 結果出力用リスト
results = []
#
for calc_date in calc_dates:
    # サンプルデータの絞り込み1: calc_date までのデータを抽出
    samples = ts_data[ts_data.index <= calc_date]
    # サンプルデータの絞り込み2:　後ろからM個のデータを抽出
    samples = samples.tail(M)
    # 平均値の計算 / pandas にやらせてしまう
    means = samples.mean()
    # 計算結果を1行のDataFrameに保存
    df = pd.DataFrame(means).T
    df.index = [calc_date]
    # 作成したDataFrameをリストに追記
    results += [df]

# 計算結果を結合して一つのDataFrameにする。
results = pd.concat(results)
print(results.head(3))
print(results.tail(3))



                    A           B
2017-02-28  99.845642  100.099281
2017-03-31  99.706632   99.710434
2017-04-30  99.807144   99.486013
                     A           B
2020-12-31  104.767165  120.075225
2021-01-31  104.602034  120.779208
2021-02-28  104.727958  121.414215


#### サンプルの絞り込み方法

- 元々のサンプルの絞り込み条件は 「基準日$t$から直近M個分のデータ」であった。
- そこで、末尾のデータを取り出す `tail` を使うと 「直近M個分」の条件をよりストレートに実装できる
- なので、上記の様な2段階の絞り込みの方がやりやすい。


#### 計算結果の `pd.DataFrame` での出力方法

- 基本的な考え方
    - `pd.DataFrame`を詰める空のリストをループ外で準備
    - 計算結果を1行分の`pd.DataFrame`に整形
    - 上記のリストに順次詰める
    - 最後に `pd.concat` で結合する。
- この書き方は割と作成者の手癖
    - 絶対の正解というわけではない。
    - あくまでやり方の一例
- 難点は繰り返しが多くなると処理が重たくなる点
    - 週ごと程度の処理なら特に問題にはならないが


**ぜひコードを分解しながら試行錯誤して理解してほしい**




## 2.4. 参考資料

移動平均のもう少し詳しい説明。

- [Wikipedia](https://ja.wikipedia.org/wiki/%E7%A7%BB%E5%8B%95%E5%B9%B3%E5%9D%87)
- [語り口の優しいサイト](https://toukeigaku-jouhou.info/2015/08/23/moving-average/)

#### ちなみに
この手合の処理は、「ローリング　統計量」や「window関数（窓関数）」などで検索すると色々な方策が見つかる。<br>
知っての通り、Pythonのfor文などは非常に計算速度が遅いので、<br>
仕事などでこの手合の分析をするときには、for文での実装ではなく　<br>
pandasなどのライブラリーの機能を活用することが望ましい。